### How to run this notebook

I used kaggle for this notebook so to run it with out any need of changing the file paths, you need to use kaggle. Make sure you have a GPU quota turned on. Then you need to clik on run all to run every cell in the notebook and wait a couple of hours as notebook runs. 

### hyperparameters used for the notebook

After loading the dataset by data loaders and building the model, the model was designed with 3 layers. we then trained the model for about 75 epochs and using learning rate of 2e-3, batch size of 64, using dropout of 0.2, lstm dropout of 0.25 and decoder dropout of 0.15, and encoder dropout of 0.3. In addition, we used CTCLoss as the criterion and we used Adam optimer for the decoder we employed CTCBeamDecoder and for the scheduler, we utilized ReducedOnPlateua with mode set to min mode and factor of 0.8, for the patience, I started with using 3 and I decreased along the running process until it eventually became 1 at the end. Gradscaler was used too for scaling. 

In [ ]:
After running the notebook the last optimal result that was obtained in 4.0095 on notebook but on making submission to kaggle the notebook scored 4.41

# Installs

In [1]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install torch-summary wandb  --quiet   #!pip install torchsummaryX wandb --quiet

In [4]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

#!pip install torchsummaryX -q
#from torchsummary import summary

fatal: destination path 'ctcdecode' already exists and is not an empty directory.
/kaggle/working/ctcdecode
/kaggle/working


In [ ]:
'''
If torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:
'''

!pip install torchsummaryx==1.1.0

# Imports

In [7]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchsummaryX import summary
#from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
#import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [8]:
# TODO: Use the same Kaggle code from HW1P2

!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"leonard250","key":"2f9461ced39b0687292591bf1315ae7a"}')
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 970.3 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73248 sha256=8bcd435dbd01cd3f5c9a4b56e274bded02c314e39b2d2c5198c53c3f8145c40c
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.6
    Uninstalling kaggle-1.6.6:
      Successfully uninstalled kaggle-1.6.6


In [9]:
!kaggle competitions download -c hw3p2asr-s24

100%|██████████████████████████████████████▉| 3.73G/3.74G [00:17<00:00, 240MB/s]
100%|███████████████████████████████████████| 3.74G/3.74G [00:17<00:00, 228MB/s]


In [10]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data
'''
!unzip -q hw3p2asr-s24.zip
!ls

11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip


In [11]:
pip install speechpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
import speechpy
import glob

# Google Drive

In [11]:
#from google.colab import drive
#drive.mount('/content/gdrive')

# Dataset and Dataloader

In [14]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

### Train Data

In [24]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,root_dir,partition,transforms,limit=None,train_460=True): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = f"{root_dir}/{partition}*/mfcc/*" #TODO.     # '*' for regex
        self.transcript_dir = f"{root_dir}/{partition}*/transcript/*"#TODO

        # self.mfcc_files = os.listdir(self.mfcc_dir) #TODO
        # self.transcript_files = os.listdir(self.mfcc_dir) #TODO

        self.mfcc_files =sorted(glob.glob(self.mfcc_dir))
        self.transcript_files= sorted(glob.glob(self.transcript_dir))

        #limit for partial data
        
        self.phonemes = PHONEMES

        assert len(self.mfcc_files) == len(self.transcript_files) 

        self.mfccs, self.transcripts = [], []

        for file in self.mfcc_files:
          self.mfccs.append(np.load(file))
        for file in self.transcript_files:
          temp =np.load(file)[1:-1]
          self.transcripts.append(np.array([self.phonemes.index(i) for i in temp]))
        
        print('before train 360 self.mfccs len =',len(self.mfccs))

        # for train 460 data
        if train_460:
          partition_460 = 'train-clean-360'
          self.mfcc_files_460= sorted(glob.glob(f"{root_dir}/{partition_460}*/mfcc/*"))
          self.transcript_files_460= sorted(glob.glob(f"{root_dir}/{partition_460}*/transcript/*"))


          for file in self.mfcc_files_460:
            self.mfccs.append(np.load(file))
          for file in self.transcript_files_460:
            temp =np.load(file)[1:-1]
            self.transcripts.append(np.array([self.phonemes.index(i) for i in temp]))
        
        print('after train 360 self.mfccs len =',len(self.mfccs))
          
        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfccs)

        assert len(self.mfccs) == len(self.transcripts)

        print('AUDIODATASET LOADED ...')
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length
        # raise NotImplemented

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = self.mfccs[ind]
        transcript = self.transcripts[ind]

        return torch.FloatTensor(mfcc), torch.tensor(transcript)
        # raise NotImplemented


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = [i[0] for i in batch] # TODO
        # batch of output phonemes
        batch_transcript = [i[1] for i in batch] # TODO

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(i) for i in batch_mfcc] # TODO 

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True) # TODO
        lengths_transcript = [len(i) for i  in batch_transcript] # TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

### Test Data

In [25]:
class AudioTestDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self,root_dir,partition,transforms,limit=None): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = f"{root_dir}/{partition}*/mfcc/*" #TODO.     # '*' for regex

        # self.mfcc_files = os.listdir(self.mfcc_dir) #TODO
        # self.transcript_files = os.listdir(self.mfcc_dir) #TODO

        self.mfcc_files =sorted(glob.glob(self.mfcc_dir))

        self.PHONEMES = PHONEMES

        self.mfccs = []

        for files in self.mfcc_files:
          self.mfccs.append(np.load(files))

        print('AUDIODATASET TEST LOADED ...')

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       

    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length
        # raise NotImplemented

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''
        mfcc = self.mfccs[ind]

        return torch.FloatTensor(mfcc)
        # raise NotImplemented


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        batch_mfcc = batch # TODO

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
        lengths_mfcc = [len(i) for i in batch_mfcc] # TODO 

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

       

### Config - Hyperparameters

In [83]:
BATCH_SIZE = 64

transforms = []


# Feel free to add more items here
config = {
    "beam_width" : 2,
    "lr"         : 2e-3,
    "epochs"     :73,
    "batch_size" : 64,  # Increase if your device can handle it
    'sc_mode'    : 'min',
    'patience'   : 1,
    'encoder dropout': 0.3,
    'decoder dropout': 0.15,
    'lstm dropout'   : 0.25,
    'sc_threshold': 1e-4,
    'factor'     : 0.8,
    "num_layers": 3,
    'threshold mode': 'rel',
    "dropout": 0.2
    
}

root = '/kaggle/working/11-785-s24-hw3p2/'

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
#transforms = [] # set of tranformations

### Data loaders

In [59]:

import gc
gc.collect()

526

In [60]:
# Create objects for the dataset class
train_data = AudioDataset(root_dir = root, partition='train-clean-100', transforms=[], limit=None,train_460=True) #TODO
val_data = AudioDataset(root_dir= root, partition='dev-clean', transforms=[], limit=None, train_460 = False ) # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioTestDataset(root_dir=root, partition='test-clean', transforms=[], limit=None) #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(dataset = train_data, num_workers=7, batch_size =config['batch_size'],pin_memory=True, shuffle=True, collate_fn= train_data.collate_fn)#TODO
val_loader = torch.utils.data.DataLoader(dataset = val_data, num_workers=4, batch_size =config['batch_size'],pin_memory=True, shuffle=True, collate_fn= val_data.collate_fn)#TODO#TODO
test_loader = torch.utils.data.DataLoader(dataset = test_data, num_workers=1, batch_size =config['batch_size'],pin_memory=True, shuffle=False, collate_fn= test_data.collate_fn)#TODO#TODO

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

before train 360 self.mfccs len = 28539
after train 360 self.mfccs len = 28539
AUDIODATASET LOADED ...
before train 360 self.mfccs len = 2703
after train 360 self.mfccs len = 2703
AUDIODATASET LOADED ...
AUDIODATASET TEST LOADED ...
Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [29]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1651, 27]) torch.Size([64, 203]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [61]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self, hidden_size, nlayers, out_size=47, embed_size=40):

        super(Network, self).__init__()
        self.nlayers = nlayers
        self.hidden_size = hidden_size
        self.embed_size = embed_size
        self.out_size = out_size
        self.cnns = torch.nn.Sequential(
            nn.Conv1d(self.embed_size, self.hidden_size, 3, padding=1, bias=False),
            nn.BatchNorm1d(self.hidden_size),
            nn.ReLU(inplace=True))
        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?
        
        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size=self.hidden_size,
                            hidden_size=self.hidden_size,
                            num_layers=3,
                            bias=True,
                            batch_first=True,
                            dropout=0.2, # regularization
                            bidirectional=True) 
       
        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            nn.Linear(self.hidden_size*2, self.hidden_size),
            nn.Linear(self.hidden_size, self.out_size)
            
        )

        
        # self.logSoftmax = nn.log_softmax(2)#TODO: Apply a log softmax here. Which dimension would apply it on ?
        self.logSoftmax =  nn.LogSoftmax(2)

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        x_cnn_input = x.permute(0, 2, 1) # (B, C_in, T_in)
        # x_cnn_input = x
        # x_cnn_input = x.transpose(1, 2)
        x_post_cnn = self.cnns(x_cnn_input) # (B, C_out, T_out)
        x_rnn_in = x_post_cnn.permute(2, 0, 1) # (T, B, C_out)
        # x_rnn_in = x_post_cnn
        # x_rnn_in = x_post_cnn.transpose(1, 2)
        x_packed = pack_padded_sequence(x_rnn_in, lx, enforce_sorted=False)
        out_packed, hidden = self.lstm(x_packed)
        out, out_lens = pad_packed_sequence(out_packed, batch_first=True) # (B, T, C)
        
        # Log softmax after output layer is required since nn.CTCLoss expect log prob
        out_prob = self.classification(out) # (B, T, Classes=47)
        out_prob = self.logSoftmax(out_prob)
        
        # Permute to fit for input format of CTCLoss
        # out_prob = out_prob.permute(1, 0, 2) #torch.transpose(out_prob, 0, 1) # (T, B, C)
        
        # TODO: calculate new xLens
        return out_prob, lx

## Initialize Basic Network
(If trying out the basic Network)

In [31]:
! pip install torch-summary

In [32]:
from torchsummary import summary


In [62]:
# Assuming your model is named Network and is already defined
model = Network(hidden_size=256, nlayers=3, out_size=len(PHONEMES), embed_size=27).to(device)
print(model)
#summary(model, x.to(device), lx)
#summary(model, input_size=(27, ))  # Replace input_size with your actual input shape


Network(
  (cnns): Sequential(
    (0): Conv1d(27, 256, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (lstm): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (classification): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=41, bias=True)
  )
  (logSoftmax): LogSoftmax(dim=2)
)


## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [63]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [64]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()
        # Initialize a single layer bidirectional LSTM with the given input_size and hidden_size
        self.blstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=config['num_layers'], batch_first=True, bidirectional=True)

    def forward(self, x_packed): # x_packed is a PackedSequence

        # Pad Packed Sequence 
        if isinstance(x_packed, torch.nn.utils.rnn.PackedSequence):
          # pad a packed sequence after computation from the RNN
          # seq_unpacked, lens_unpacked = pad_packed_sequence(packed, batch first=True)
          x, x_lens = pad_packed_sequence(x_packed, batch_first=True)
        else:
          x = x_packed
          x_lens = None

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        x, x_lens = self.trunc_reshape(x, x_lens)
        
        # Pack Padded Sequence. What output(s) would you get?
        if x_lens is not None:
          # pack a padded sequence before passing it through an RNN
          # packed = pack_padded_sequence(recordings, length_of_each_recording, batch_first=True, enforce_sorted=False)
          x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
        else:
          x_packed = x

        # Pass the sequence through bLSTM
        output, (h_n, c_n) = self.blstm(x_packed)

        # What do you return?
        return output

    def trunc_reshape(self, x, x_lens): 
        batch_size, length, features = x.shape
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        '''
        If the length of the original input sequence is odd, then it is either padded out by appending
        an extra vector of zeros at the end, before reshaping, or trimmed to an even length by deleting
        the final vector. 
        '''
        if (length % 2) != 0:
          # trim to an even length by deleting the final vector and decrementing length
          x = x[:, :-1, :]
          x_lens[-1] -= 1
        
        downsampled_length = length // 2
        # Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        x = torch.reshape(x, shape=(batch_size, downsampled_length, 2 * features))
        # Reduce lengths by the same downsampling factor = 2
        x_lens = torch.clamp(x_lens, max=downsampled_length, out=None)
        return x, x_lens

In [65]:
from torch.autograd import Variable
class LockedDropout(nn.Module):
  '''
  Locked dropout is a variation of dropout that is specifically designed for use in 
  the parallel bidirectional LSTM (pBLSTM) architecture. It uses the same dropout mask 
  for both the forward and backward layers.  This ensures that the same set of neurons 
  are dropped out at each time step for both layers, maintaining consistency across time.

  From https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py
  '''
  def __init__(self):
    super().__init__()

  def forward(self, x, dropout=config['dropout']):
    if not self.training or not dropout:
      return x
    x, x_lens = pad_packed_sequence(x, batch_first=True)
    m = x.new(x.shape[0],1,x.shape[2]).bernoulli_(1-dropout)
    mask = Variable(m, requires_grad=False) / (1 - dropout)
    mask = mask.expand_as(x)
    x = mask * x
    # pack a padded sequence using before passing it through an RNN
    x_packed = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)
    return x_packed

### Encoder

In [37]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, embed_size, hidden_size):
        super(Encoder, self).__init__()

        # You can use CNNs as Embedding layer to extract features. 
        # Keep in mind the Input dimensions and expected dimension of Pytorch CNN.
        self.embedding = torch.nn.Sequential(
            PermuteBlock(), # transpose to (N, C, L) for Conv1d
            nn.Conv1d(input_size, embed_size, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm1d(embed_size),
            PermuteBlock(), # transpose back for RNN
        )
        # How many pBLSTMs are required?
        # Fill this up with pBLSTMs - What should the input_size be? 
        # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
        self.pBLSTMs = torch.nn.Sequential(
            pBLSTM(input_size=2*embed_size, hidden_size=hidden_size),
            LockedDropout(),
            pBLSTM(input_size=4*hidden_size, hidden_size=hidden_size),
            LockedDropout(),
        )          
    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        
        # Call the embedding layer
        x = self.embedding(x)
        
        # Pack padded sequence
        x = pack_padded_sequence(x, x_lens, batch_first=True, enforce_sorted=False)

        # Pass Sequence through the pyramidal Bi-LSTM layer
        x = self.pBLSTMs(x)

         # Pad packed sequence
        encoder_outputs, encoder_lens = pad_packed_sequence(x, batch_first=True)

        return encoder_outputs, encoder_lens

### Decoder

In [66]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(2 * embed_size), PermuteBlock(),
            #Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(2*embed_size, 2048),
            PermuteBlock(), torch.nn.BatchNorm1d(2048), PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Linear(2048, 2048),
            PermuteBlock(), torch.nn.BatchNorm1d(2048), PermuteBlock(),
            torch.nn.GELU(),
            torch.nn.Dropout(config['dropout']),
            torch.nn.Linear(2048, output_size)
        )
        
        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        # call your MLP
        out = self.mlp(encoder_out)
        # Think what should be the final output of the decoder for the classification 
        out = self.softmax(out)

        return out 

In [67]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size, hidden_size, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations = torch.nn.Sequential(
            # Add Time Masking/ Frequency Masking
            # Hint: See how to use PermuteBlock() function defined above
            PermuteBlock(),
            tat.FrequencyMasking(freq_mask_param=10),
            tat.TimeMasking(time_mask_param=40),
            PermuteBlock(),
        )
        # Initialize Encoder
        self.encoder = Encoder(input_size=input_size, embed_size = embed_size, hidden_size=hidden_size)
        # Initialize Decoder
        self.decoder = Decoder(embed_size=hidden_size, output_size=output_size) 
        
    
    def forward(self, x, lengths_x):
        if self.training:
            x = self.augmentations(x)
        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [68]:
model = ASRModel(
    input_size  = 27, #TODO,
    embed_size  =  27, #TODO
    hidden_size= 256,
    output_size = len(PHONEMES)
).to(device)
print(model)
#summary(model, x.to(device), lx)
#summary(model, input_size=(27, None)) 

ASRModel(
  (augmentations): Sequential(
    (0): PermuteBlock()
    (1): FrequencyMasking()
    (2): TimeMasking()
    (3): PermuteBlock()
  )
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(27, 27, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (2): BatchNorm1d(27, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): PermuteBlock()
    )
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(54, 256, num_layers=3, batch_first=True, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (blstm): LSTM(1024, 256, num_layers=3, batch_first=True, bidirectional=True)
      )
      (3): LockedDropout()
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=512, out_features=2048, bias=True)
      

# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [84]:
#TODO


criterion = nn.CTCLoss(blank= CMUdict.index(''))# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

# optimizer =  torch.optim.AdamW(...) # What goes in here?
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=LABELS, beam_width=config['beam_width'], num_processes= 4, log_probs_input=True)#TODO 

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= config['sc_mode'], factor= config['factor'], patience= config['patience'], threshold= config['sc_threshold'], threshold_mode= config['threshold mode'],  verbose=True)#TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [85]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):
    
    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    output = torch.transpose(output, 0, 1) 
    output, _, _, out_seq_len = decoder.decode(output, seq_lens= output_lens) #lengths - list of lengths

    pred_strings = []
    
    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        logits=output[i][0][:out_seq_len[i][0]]
        # print('index',i)
        # print('logits',max(logits))
        try: pred_strings.append(''.join([PHONEME_MAP[i] for i in logits]))
        except: 
          print(logits)
          return
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers
    
    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    # print(pred_strings)
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]#TODO
        label_string = ''.join(PHONEME_MAP[l] for l in label[i][:label_lens[i]])#TODO
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

# Test Implementation

In [86]:
model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 573, 41])
torch.Size([573, 64, 41]) torch.Size([64, 241])
tensor(0.3120, device='cuda:0', grad_fn=<MeanBackward0>)
3.140625


# WandB

You will need to fetch your api key from wandb.ai

In [72]:
import wandb
wandb.login(key="af28e15472d2b0ef281fdacbb101cc157ad1b80d") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [73]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    #reinit = True, ### Allows reinitalizing runs when you re-run this cell
    id = "av362bvm", #Insert specific run id here if you want to resume a previous run
    resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "HW3P2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

# Train Functions

## Training Setup

In [74]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):
    
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():     
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss 
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar
    
    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        # vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)
        vdist += calculate_levenshtein(h, y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()
    
        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()
        
    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

In [87]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 664, 41])
torch.Size([664, 64, 41]) torch.Size([64, 260])
tensor(0.3418, device='cuda:0', grad_fn=<MeanBackward0>)
4.25


In [76]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1], 
         'epoch'                    : epoch}, 
         path
    )

def load_model(path, model, metric= 'valid_dist', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [77]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
epoch_model_path = "/kaggle/working/checkpoint.pth"#TODO set the model path( Optional, you can just store best one. Make sure to make the changes below )
best_model_path = "/kaggle/working/best_checkpoint.pth"#TODO set best model path 

In [89]:

#TODO: Please complete the training loop
def experiment(start_epoch=0):
    torch.cuda.empty_cache()
    gc.collect()
    best_lev_dist = float('inf')
    
    for epoch in range(start_epoch, config['epochs']):

        print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))
    
        curr_lr = float(optimizer.param_groups[0]['lr']) #TODO

        train_loss              = train_model(model, train_loader, criterion, optimizer)#TODO
        valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)#TODO
        scheduler.step(valid_dist)

        print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
        print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


        wandb.log({
            'train_loss': train_loss,  
            'valid_dist': valid_dist, 
           'valid_loss': valid_loss, 
           'lr'        : curr_lr
            })
    
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
        wandb.save(epoch_model_path)
        print("Saved epoch model")

        if valid_dist <= best_lev_dist:
            best_lev_dist = valid_dist
            save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
            wandb.save(best_model_path)
            print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
    run.finish()

In [90]:
#wandb.init(project="hw2p2 ablations", entity="01 face verification & face classification")

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    epoch = checkpoint['epoch']
    val_dist = checkpoint['valid_dist']
    return model, optimizer, scheduler, epoch, val_dist


# Load the checkpoint if available
start_epoch = 0
checkpoint_val_dist = './best_checkpoint.pth'


if os.path.exists(checkpoint_val_dist):
    print("loading checkpoint...")
    model, optimizer, scheduler, starting_epoch, best_class_acc = load_checkpoint(checkpoint_val_dist )
    experiment(start_epoch= starting_epoch)
else:
    print("the checkpoint paths does not exist")
    print("starting from scratch")
    experiment()


loading checkpoint...



Epoch: 73/73


Epoch 00014: reducing learning rate of group 0 to 6.1035e-08.
	Train Loss 0.0539	 Learning Rate 0.0000001
	Val Dist 3.9983%	 Val Loss 0.2988
Saved epoch model
Saved best model


wandb: WARNING Ensure read and write access to run files dir: /kaggle/working/wandb/run-20240323_052425-av362bvm/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


lr,██▄▄▂▂▁▄
train_loss,█▅▅▁▃▆█▇
valid_dist,▃▂▁█▄▆▇▅
valid_loss,▆█▅▁▇▆█▂
lr,0.0
train_loss,0.05388
valid_dist,3.99826
valid_loss,0.29879


In [59]:
try:
    artifact1 = wandb.restore("best_checkpoint.pth")
    gc.collect()
    wandb.watch(model, log="all")
    # Check if the artifact is valid
    if artifact1 is not None:
        best_val_acc = 0.0
        checkpoint_path = artifact1.name

            # Load checkpoint if exists
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint.get('epoch', 1)
            best_val_acc = checkpoint.get('val_acc', 0)
            experiment(start_epoch= start_epoch)
            


        else:
            start_epoch = 1
            best_val_acc = 0.0
except Exception as ex:
    print(f"Exception is thrown because {ex}")
    pass


Epoch: 62/70


	Train Loss 0.0530	 Learning Rate 0.0000039
	Val Dist 3.9909%	 Val Loss 0.3047


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved epoch model
Saved best model

Epoch: 63/70


	Train Loss 0.0512	 Learning Rate 0.0000039
	Val Dist 4.0232%	 Val Loss 0.3081
Saved epoch model

Epoch: 64/70


...

	Train Loss 0.0570	 Learning Rate 0.0000020
	Val Dist 3.9663%	 Val Loss 0.3094
Saved epoch model

Epoch: 69/70


	Train Loss 0.0510	 Learning Rate 0.0000020
	Val Dist 3.9583%	 Val Loss 0.3051
Saved epoch model
Saved best model

Epoch: 70/70


	Train Loss 0.0570	 Learning Rate 0.0000020
	Val Dist 4.0104%	 Val Loss 0.3137
Saved epoch model


wandb: WARNING Ensure read and write access to run files dir: /kaggle/working/wandb/run-20240322_204448-av362bvm/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


lr,███▁▁▁▁▁▁
train_loss,▅▄▅▅▁▄█▃█
valid_dist,▅█▆▂▇▁▂▁▇
valid_loss,▂▃▇▁▃█▄▂▅
lr,0.0
train_loss,0.05698
valid_dist,4.01044
valid_loss,0.31369


In [52]:
try:
    artifact1 = wandb.restore("best_checkpoint.pth")
    gc.collect()
    wandb.watch(model, log="all")
    # Check if the artifact is valid
    if artifact1 is not None:
        best_val_acc = 0.0
        checkpoint_path = artifact1.name

            # Load checkpoint if exists
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint.get('epoch', 1)
            best_val_acc = checkpoint.get('val_acc', 0)
            experiment(start_epoch= start_epoch)
            


        else:
            start_epoch = 1
            best_val_acc = 0.0
except Exception as ex:
    print(f"Exception is thrown because {ex}")
    pass


Epoch: 62/75


	Train Loss 0.0529	 Learning Rate 0.0000039
	Val Dist 3.9690%	 Val Loss 0.3101


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved epoch model
Saved best model

Epoch: 63/75


	Train Loss 0.0515	 Learning Rate 0.0000039
	Val Dist 3.9879%	 Val Loss 0.3031
Saved epoch model

Epoch: 64/75


...

Epoch 00012: reducing learning rate of group 0 to 1.2207e-07.
	Train Loss 0.0480	 Learning Rate 0.0000002
	Val Dist 3.9872%	 Val Loss 0.3051
Saved epoch model

Epoch: 74/75


	Train Loss 0.0494	 Learning Rate 0.0000001
	Val Dist 3.9751%	 Val Loss 0.3134
Saved epoch model

Epoch: 75/75


Epoch 00014: reducing learning rate of group 0 to 6.1035e-08.
	Train Loss 0.0539	 Learning Rate 0.0000001
	Val Dist 4.0059%	 Val Loss 0.2949
Saved epoch model


wandb: WARNING Ensure read and write access to run files dir: /kaggle/working/wandb/run-20240323_034315-av362bvm/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


lr,███▄▄▃▃▂▂▁▁▁▁▁
train_loss,▆▅▁▄▄▅▅█▁▇▇▁▃▇
valid_dist,▂▄▃▄▄█▅▇▃▁▄▄▂▆
valid_loss,▆▄▆█▅█▁▄▆▅▃▅▇▂
lr,0.0
train_loss,0.05388
valid_dist,4.00589
valid_loss,0.29493


# Generate Predictions and Submit to Kaggle

In [91]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 2 #TODO

test_decoder    = CTCBeamDecoder(labels=LABELS, beam_width=TEST_BEAM_WIDTH, log_probs_input=True)#TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)
        h = torch.permute(h, (1, 0, 2))
    prediction_string= decode_prediction(h, lh, test_decoder, LABELS)# TODO call decode_prediction 
    #TODO save the output in results array.
    results.extend(prediction_string)
    
    del x, lx, h, lh
    torch.cuda.empty_cache()


Testing



100%|██████████| 41/41 [00:26<00:00,  1.55it/s]


In [55]:
import pandas as pd

In [56]:
#'/kaggle/working/11-785-s24-hw3p2/'
# data_dir = "/kaggle/working/11-785-s24-hw3p2/" + "/test-clean/random_submission.csv"
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

In [92]:
!kaggle competitions submit -c hw3p2asr-s24 -f submission.csv -m "I made it!"

100%|█████████████████████████████████████████| 209k/209k [00:01<00:00, 206kB/s]
Successfully submitted to HW3P2_ASR-S24

In [ ]:
# Run this cell to download the file
!kaggle competitions download -c hw3p2asr-s24 -f test-clean/random_submission.csv

# The downloaded file will be saved in the current directory
